<a href="https://colab.research.google.com/github/lakshmansamvith/Kaggle-Compete-/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded_file = files.upload()

Saving train.csv to train.csv
Saving test.csv to test.csv


#Read CSV Files 

In [0]:
import pandas as pd 
import numpy as np 
pd.set_option("display.max_rows", 50, "display.max_columns", None)
training_data_frame = pd.read_csv('train.csv')
testing_data_frame  = pd.read_csv('test.csv')
#testing_data_frame

#Check for NaN values (Data Preprocessing) and Drop Columns with more than 75% Null Values 

In [297]:
train_nan_values = list()
for column in training_data_frame.columns:
    train_nan_values.append(training_data_frame[column].isnull().sum())

train_null_ratio_index = list()
for value in train_nan_values: 
    if value/len(training_data_frame) > 0.75:
       train_null_ratio_index.append(train_nan_values.index(value))
print(train_null_ratio_index)

test_nan_values = list()
for column in testing_data_frame.columns:
        test_nan_values.append(testing_data_frame[column].isnull().sum())
        
test_null_ratio_index = list()
for value in test_nan_values: 
    if value/len(testing_data_frame) > 0.75:
       test_null_ratio_index.append(test_nan_values.index(value))


print(train_nan_values, test_nan_values)
print(train_null_ratio_index , test_null_ratio_index )

for elem in test_null_ratio_index:
    testing_data_frame = testing_data_frame.drop(testing_data_frame.columns[test_null_ratio_index], axis=1)

for elem in train_null_ratio_index:
    training_data_frame = training_data_frame.drop(training_data_frame.columns[train_null_ratio_index], axis=1)

#testing_data_frame

[10]
[0, 0, 0, 0, 0, 177, 0, 0, 0, 0, 687, 2] [0, 0, 0, 0, 86, 0, 0, 0, 1, 327, 0]
[10] [9]



# Label Encoding 


In [0]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

training_data_frame['Embarked'] = training_data_frame['Embarked'].fillna('')
testing_data_frame['Embarked'] = testing_data_frame['Embarked'].fillna('')

training_categorical_values = [training_data_frame['Sex'].values, training_data_frame['Ticket'].values, training_data_frame['Embarked'].values ]
for num in range(len(training_categorical_values)):
        #print(training_categorical_values[num])
        training_categorical_values[num] = label_encoder.fit_transform(training_categorical_values[num])

testing_categorical_values = [testing_data_frame['Sex'].values, testing_data_frame['Ticket'].values, testing_data_frame['Embarked'].values]
for num in range(len(testing_categorical_values)):
        testing_categorical_values[num] = label_encoder.fit_transform(testing_categorical_values[num])    
    
training_data_frame['Sex'], training_data_frame['Ticket'], training_data_frame['Embarked'] = training_categorical_values

testing_data_frame['Sex'], testing_data_frame['Ticket'], testing_data_frame['Embarked'] = testing_categorical_values





#Interpolating all the Missing Values in the DataFrames 

In [0]:
training_data_frame = training_data_frame.interpolate(method ='polynomial', order=5)
testing_data_frame = testing_data_frame.interpolate(method ='polynomial', order=5) 


#Feature Engineering(Creating Two New Features)

In [0]:
training_data_frame['total_family'] = training_data_frame['Parch'] + training_data_frame['SibSp']
training_data_frame['Individual_Fare'] = training_data_frame['Fare']/(training_data_frame['total_family']+1)
training_data_frame['Age*Class'] = training_data_frame['Age']*training_data_frame['Pclass'] 


testing_data_frame['total_family'] = testing_data_frame['Parch'] + testing_data_frame['SibSp']
testing_data_frame['Individual_Fare'] = testing_data_frame['Fare']/(testing_data_frame['total_family']+1)
testing_data_frame['Age*Class'] =  testing_data_frame['Age']*testing_data_frame['Pclass']

#training_data_frame


# Feature Correlations (Establishing Feature and Target Variables) 

In [300]:
correlations = training_data_frame.corr()
survived_corr = correlations['Survived']
count = 0
print(survived_corr)

required_attributes = list()
for elem in survived_corr: 
    if elem >0.20: 
       required_attributes.append(correlations.columns[count])
    count = count + 1  

target_attributes, featured_attributes = required_attributes[:1], required_attributes[1:] 
print(featured_attributes, target_attributes)


PassengerId       -0.005007
Survived           1.000000
Pclass            -0.338481
Age               -0.037111
SibSp             -0.035322
Parch              0.081629
Fare               0.257307
total_family       0.016639
Individual_Fare    0.221600
Age*Class         -0.229061
Name: Survived, dtype: float64
['Fare', 'Individual_Fare'] ['Survived']


# Splitting into Feature and Target values(Training Data)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(training_data_frame[featured_attributes].values, 
                                                    training_data_frame[target_attributes].values.reshape(len(training_data_frame),), 
                                                    random_state=0)

#Scaling Attributes 

In [0]:
from sklearn.preprocessing import RobustScaler
scaler =   RobustScaler()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


# Building a Gaussian Classifier  Model 

In [303]:
from sklearn.naive_bayes import GaussianNB 
gaussian_model = GaussianNB()
gaussian_model.fit(X_train, y_train)
y_predicted_values = gaussian_model.predict(X_test)
print(gaussian_model.score(X_test, y_test))

0.6995515695067265


#Building a RandomForest Classifier 

In [304]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(criterion='gini', max_depth=5 )
rf_clf.fit(X_train, y_train)
print(rf_clf.score(X_test, y_test))

0.726457399103139


#Building a DecisionTreeClassifier

In [305]:
from sklearn.tree import DecisionTreeClassifier 
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train, y_train)
print(dt_clf.score(X_test, y_test))

0.6995515695067265


#Building a Logistic Regression Model 

In [306]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.695067264573991

#Building a Gradient Booster Classifier 

In [307]:
from sklearn.ensemble import GradientBoostingClassifier 
gb_clf = GradientBoostingClassifier()
gb_clf.fit(X_train, y_train)
gb_clf.score(X_test, y_test)


0.7488789237668162

#Choosing the best Model and creating the model for Train Set

In [0]:
X_train, y_train = training_data_frame[featured_attributes].values, training_data_frame[target_attributes].values.reshape(len(training_data_frame),)
X_test = testing_data_frame[featured_attributes]
scaler = Normalizer()
scaler = scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
gb_clf1 = GradientBoostingClassifier()
gb_clf1.fit(X_train, y_train)
predicted_values = gb_clf1.predict(X_test).tolist()


In [0]:
import  csv
#(testing_data_frame)
Passenger_Ids = testing_data_frame['PassengerId'].values.tolist()
row = 0
with open('Answer.csv', 'w') as file:
     writer = csv.writer(file, lineterminator='\n')
     for count in range(len(Passenger_Ids)): 
         if row == 0:
             writer.writerow(['PassengerId', 'Survived'])
         writer.writerow([Passenger_Ids[count], predicted_values[count]])
         row = row + 1